In [1]:
import pyodbc
import subprocess
import os
import shutil
import pandas as pd
import csv
from getpass import getuser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Edge()
driver.get("https://www.spensiones.cl/apps/centroEstadisticas/paginaCuadrosCCEE.php?menu=sci&menuN1=cotycot&menuN2=ingimp")

In [4]:
driver.find_element(By.XPATH, "/html/body/div[2]/div/div[3]/div/div/form/table/tbody/tr[2]/td[3]/input").click()

In [ ]:
input_year = 1
input_month = 2

tbody = driver.find_element(By.XPATH, "/html/body/div[4]/div/div[2]/div/div[2]/div[2]/div//table")
data = []
for td in tbody.find_elements(By.XPATH, "//tr[1]"):
    row = [i.text for i in td.find_elements(By.TAG_NAME, "td")]
    data.append(row)
data.pop(0)
for td in tbody.find_elements(By.XPATH, "//tr[2]"):
    row = [i.text.replace(".","") for i in td.find_elements(By.TAG_NAME, "td")]
    data.append(row)
data.pop(1)
for td in tbody.find_elements(By.XPATH, "//tr[3]"):
    row = [i.text.replace(".","") for i in td.find_elements(By.TAG_NAME, "td")]
    data.append(row)
data.pop(2)
for td in tbody.find_elements(By.XPATH, "//tr[4]"):
    row = [i.text.replace(".","") for i in td.find_elements(By.TAG_NAME, "td")]
    data.append(row)
for td in tbody.find_elements(By.XPATH, "//tr[5]"):
    row = [i.text.replace(".","") for i in td.find_elements(By.TAG_NAME, "td")]
    data.append(row)
for td in tbody.find_elements(By.XPATH, "//tr[6]"):
    row = [i.text.replace(".","") for i in td.find_elements(By.TAG_NAME, "td")]
    data.append(row)
for td in tbody.find_elements(By.XPATH, "//tr[7]"):
    row = [i.text.replace(".","") for i in td.find_elements(By.TAG_NAME, "td")]
    data.append(row)
for td in tbody.find_elements(By.XPATH, "//tr[8]"):
    row = [i.text.replace(".","") for i in td.find_elements(By.TAG_NAME, "td")]
    data.append(row)


user_id = getuser()

In [ ]:
Ruta = r'C:/Users/{}/Metlife/Sharepoint Chile Seguros - PLANNING/Bases/Cloud/Archivos/Entrada/DtPb_SP_IngresoImponiblePromedio/{}.csv'.format(user_id,input_year+input_month)

with open (Ruta, 'a', newline = "") as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerows(data)

In [ ]:
project_id = 'v13513clinsfindataczprod00001'
dataset_id = 'bgqbi_finance001'
table_id = 'DtPb_SP_IngresoImponiblePromedio'
leading_rows = 0
field_delimiter = ";"
repl_carga = "false"
estruc_json = r'C:/Users/{}/Metlife/Sharepoint Chile Seguros - PLANNING/Bases/Cloud/Estructuras/{}.json'.format(user_id,table_id)
local_path_input = r'C:/Users/{}/Metlife/Sharepoint Chile Seguros - PLANNING/Bases/Cloud/Archivos/Entrada/'.format(user_id)
local_path_done = r'C:/Users/{}/Metlife/Sharepoint Chile Seguros - PLANNING/Bases/Cloud/Archivos/Procesado/'.format(user_id)
local_path_root = r'C:/temp'
local_file = ""
local_file_root = ""

shutil.copy(estruc_json,r'{}{}.json'.format(local_path_root,table_id))

str_acceso = "gcloud auth login"
process = subprocess.run(str_acceso, shell = True, capture_output = True)
returncode = (process.returncode)
mensaje = str(process.stdout)+str(process.stderr)
print(returncode,mensaje.replace('\\','/'))

local_path = r'{}{}/'.format(local_path_input,table_id)
for file in os.listdir(local_path):
    print(file)

    local_file_root = r'{}{}'.format(local_path_root,file)
    local_file = r'{}{}'.format(local_path,file)
    local_path_done = r'{}{}/{}'.format(local_path_done,table_id,file)

    shutil.copy(local_file,local_file_root)

    gs_path = r'gs://strczclinsfin001/Inputs/{}/{}'.format(table_id,file)

    carga_storage = 'gsutil cp {} {}'.format(local_file_root,gs_path)
    process = subprocess.run(carga_storage, shell = True, capture_output = True)
    returncode = (process.returncode)
    mensaje = str(process.stdout)+str(process.stderr)

    carga_bq = 'bq load --skip_leading_rows={} --field_delimiter="{}" --replace={} {}.{} {} {}'.format(leading_rows,field_delimiter,repl_carga,dataset_id,table_id,gs_path,r'{}{}.json'.format(local_path_root,table_id))
    process = subprocess.run(carga_bq, shell = True, capture_output = True)
    returncode = (process.returncode)
    mensaje = str(process.stdout)+str(process.stderr)
    print(returncode,mensaje.replace('\\','/'))

    os.remove(local_file_root)
    shutil.move(local_file,local_path_done)

os.remove(r'{}{}.json'.format(local_path_root,table_id))